In [1]:
# Imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from datetime import datetime, timedelta

# import sqlite3
# from airflow import DAG
# from airflow.utils.dates import days_ago
# from airflow.operators.python_operator import PythonOperator

import nbformat

import warnings
from typing import Optional, Tuple, Callable, Dict, Any, List, Union
import fire

warnings.filterwarnings('ignore')

# Create a requirements.txt file with the necessary packages
# !pip freeze > airflow/dags/requirements.txt

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet, Lasso, Ridge, SGDRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.discriminant_analysis import StandardScaler

from fastapi import FastAPI

import pickle
import os


/var/folders/3g/lt92y6v536s5sxynst_qrtsm0000gn/T/ipykernel_37760/464416983.py:23 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [3]:
from src.load_data import load_data_from_source
from src.transform_data import transform_data_into_features_and_targets
from src.baseline_model import train_baseline

from src.train_models import train_model, save_best_model, training_process

from src.model_prediction import predict_res

from src.execution import predict, root

In [4]:
# ---- VARIABLES ----
explanable_cols = ['Player','Nation','Pos','Squad','Age','Born','Starts','Min','Gls','Total_Att','Blocks_Blocks','Blocks_Sh','Blocks_Pass','Clr','Err','Touches_Touches','Touches_DefPen','Dribbles_Succ','Dribbles_Att','Dribbles_Mis','AerialDuels_Won','AerialDuels_Lost']
file_loc       = 'airflow/dags/assets/matches-checkpoint.csv'
spanish_squads = ['Sevilla', 'Sporting Huelva', 'Athletic Club', 'Levante Planas',
                  'UDG Tenerife', 'Villarreal', 'Madrid CFF', 'Barcelona',
                  'Atlético Madrid', 'Real Madrid', 'Alhama', 'Alavés',
                  'Real Sociedad', 'Levante', 'Real Betis', 'Valencia']

## Load Data

In [5]:
data = load_data_from_source()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Wk         306 non-null    int64  
 1   Day        306 non-null    object 
 2   Date       306 non-null    object 
 3   Time       306 non-null    object 
 4   Home       306 non-null    object 
 5   xGHome     305 non-null    float64
 6   Score      306 non-null    object 
 7   xGAway     305 non-null    float64
 8   Away       306 non-null    object 
 9   Home_id    306 non-null    object 
 10  Away_id    306 non-null    object 
 11  Match_id   306 non-null    object 
 12  League_id  306 non-null    int64  
 13  xPHome     306 non-null    float64
 14  xPAway     306 non-null    float64
 15  ScoreHome  306 non-null    int64  
 16  ScoreAway  306 non-null    int64  
dtypes: float64(4), int64(4), object(9)
memory usage: 40.8+ KB
(306, 17) None


,Wk,Day,Date,Time,Home,xGHome,Score,xGAway,Away,Home_id,Away_id,Match_id,League_id,xPHome,xPAway,ScoreHome,ScoreAway
0,2,Sat,2022-09-17,12:00,Barcelona,1.9,2–0,0.4,UDG Tenerife,15f49df1,4c088abe,4df3a732,230,2.40,0.40,2,0
1,2,Sat,2022-09-17,12:00,Alavés,1.1,1–2,1.2,Madrid CFF,aa11fb42,89818574,87c755cd,230,1.29,1.43,1,2
2,2,Sat,2022-09-17,16:00,Real Sociedad,0.7,2–0,0.3,Villarreal,c21f25d3,7a7bef84,abfde9d9,230,1.68,0.87,2,0
3,2,Sat,2022-09-17,16:00,Real Madrid,1.6,2–0,0.8,Valencia,54582b93,f96ff499,d0329f46,230,1.94,0.80,2,0
4,2,Sat,2022-09-17,18:00,Sevilla,1.1,1–3,1.4,Atlético Madrid,215d9026,b56c2667,f4452586,230,1.15,1.58,1,3


## Transform Data

Transform the loaded data into _features_ and _target_


In [6]:
features_home, target_home = transform_data_into_features_and_targets(df=data, score='ScoreHome')
features_away, target_away = transform_data_into_features_and_targets(df=data, score='ScoreAway')

# Remember that features_home and features_away are the same, so we can use either one
# However, target_home and target_away are different, so we need to concatenate them in order to have the full target

# Modelling

## Baseline Model
Establish a baseline performance against which to compare future better models

In [7]:
train_baseline(features_home, target_home)
train_baseline(features_away, target_away)

Baseline error: 1.1899063475546305
Baseline error: 1.1488033298647242


## Other Models

In [8]:

rankings_home = {}
training_process(rankings_home, features_home, target_home)
# train_model(rankings_home,'LinearRegression', features_home, target_home)
# train_model(rankings_home,'AdaBoostRegressor', features_home, target_home)
# train_model(rankings_home,'RandomForestRegressor', features_home, target_home)
# train_model(rankings_home,'XGBRegressor', features_home, target_home)
# # train_model(rankings_home,'DecisionTreeRegressor', features_home, target_home)
# # train_model(rankings_home,'KNeighborsRegressor', features_home, target_home)
# # train_model(rankings_home,'GradientBoostingRegressor', features_home, target_home)
# # train_model(rankings_home,'Ridge', features_home, target_home)
# # train_model(rankings_home,'Lasso', features_home, target_home)
# # train_model(rankings_home,'ElasticNet', features_home, target_home)
# # train_model(rankings_home,'SGDRegressor', features_home, target_home)
# # train_model(rankings_home,'KernelRidge', features_home, target_home)

# print(sorted(rankings_home.items(), key=lambda x: x[1]['mae']))

# Save models and their info
save_best_model('home',rankings_home)

                                                model  \
0                       Ridge(alpha=1, solver='lsqr')   
1           LinearRegression(n_jobs=1, positive=True)   
2   KernelRidge(alpha=10, degree=1, gamma=1, kerne...   
3   SGDRegressor(alpha=0.1, learning_rate='adaptiv...   
4   XGBRegressor(base_score=None, booster=None, ca...   
5   (DecisionTreeRegressor(max_depth=2, max_featur...   
6   ([DecisionTreeRegressor(criterion='friedman_ms...   
7   (DecisionTreeRegressor(max_depth=3, random_sta...   
8                  DecisionTreeRegressor(max_depth=2)   
9                 KNeighborsRegressor(n_neighbors=10)   
10                  ElasticNet(alpha=1, l1_ratio=0.1)   
11                                     Lasso(alpha=1)   

                                               params       mae  \
0   {'alpha': 1, 'fit_intercept': True, 'solver': ...  0.827189   
1   {'fit_intercept': True, 'n_jobs': 1, 'positive...  0.832842   
2   {'alpha': 10, 'degree': 1, 'gamma': 1, 'kernel...  0.

,model,params,mae,time
0,"Ridge(alpha=1, solver='lsqr')","{'alpha': 1, 'fit_intercept': True, 'solver': ...",0.827189,0 days 00:00:01.152000
1,"LinearRegression(n_jobs=1, positive=True)","{'fit_intercept': True, 'n_jobs': 1, 'positive...",0.832842,0 days 00:00:00.517935
2,"KernelRidge(alpha=10, degree=1, gamma=1, kerne...","{'alpha': 10, 'degree': 1, 'gamma': 1, 'kernel...",0.838041,0 days 00:00:20.506329
3,"SGDRegressor(alpha=0.1, learning_rate='adaptiv...","{'alpha': 0.1, 'fit_intercept': True, 'learnin...",0.857431,0 days 00:00:18.930050
4,"XGBRegressor(base_score=None, booster=None, ca...","{'max_depth': 2, 'n_estimators': 50}",0.886967,0 days 00:00:45.143286
5,"(DecisionTreeRegressor(max_depth=2, max_featur...","{'max_depth': 2, 'n_estimators': 100}",0.890985,0 days 00:01:08.059098
6,([DecisionTreeRegressor(criterion='friedman_ms...,"{'max_depth': 2, 'n_estimators': 50}",0.891445,0 days 00:00:39.691355
7,"(DecisionTreeRegressor(max_depth=3, random_sta...","{'learning_rate': 0.01, 'n_estimators': 50}",0.893049,0 days 00:00:39.782362
8,DecisionTreeRegressor(max_depth=2),{'max_depth': 2},0.923438,0 days 00:00:00.232854
9,KNeighborsRegressor(n_neighbors=10),{'n_neighbors': 10},0.975806,0 days 00:00:00.209279


In [9]:
rankings_away = {}
training_process(rankings_away, features_away, target_away)
# train_model(rankings_away, 'LinearRegression', features_away, target_away)
# train_model(rankings_away, 'AdaBoostRegressor', features_away, target_away)
# train_model(rankings_away, 'RandomForestRegressor', features_away, target_away)
# train_model(rankings_away, 'XGBRegressor', features_away, target_away)
# train_model(rankings_away, 'DecisionTreeRegressor', features_away, target_away)
# train_model(rankings_away, 'KNeighborsRegressor', features_away, target_away)
# train_model(rankings_away, 'GradientBoostingRegressor', features_away, target_away)
# train_model(rankings_away, 'Ridge', features_away, target_away)
# train_model(rankings_away, 'Lasso', features_away, target_away)
# train_model(rankings_away, 'ElasticNet', features_away, target_away)
# train_model(rankings_away, 'SGDRegressor', features_away, target_away)
# train_model(rankings_away, 'KernelRidge', features_away, target_away)

print(sorted(rankings_away.items(), key=lambda x: x[1]['mae']))

# Save models and their info
save_best_model('away',rankings_away)

[('Ridge', {'model': Ridge(alpha=1, solver='lsqr'), 'params': {'alpha': 1, 'fit_intercept': True, 'solver': 'lsqr'}, 'mae': 0.8271892857909143, 'time': datetime.timedelta(seconds=1, microseconds=566549)}), ('LinearRegression', {'model': LinearRegression(n_jobs=1, positive=True), 'params': {'fit_intercept': True, 'n_jobs': 1, 'positive': True}, 'mae': 0.8328416830147468, 'time': datetime.timedelta(microseconds=601017)}), ('KernelRidge', {'model': KernelRidge(alpha=10, degree=1, gamma=1, kernel='poly'), 'params': {'alpha': 10, 'degree': 1, 'gamma': 1, 'kernel': 'poly'}, 'mae': 0.8380411369943264, 'time': datetime.timedelta(seconds=24, microseconds=954340)}), ('SGDRegressor', {'model': SGDRegressor(alpha=0.1, learning_rate='adaptive', loss='epsilon_insensitive',
             penalty='l1'), 'params': {'alpha': 0.1, 'fit_intercept': True, 'learning_rate': 'adaptive', 'loss': 'epsilon_insensitive', 'penalty': 'l1', 'shuffle': True}, 'mae': 0.8602809591842592, 'time': datetime.timedelta(secon

,model,params,mae,time
0,"Ridge(alpha=1, solver='lsqr')","{'alpha': 1, 'fit_intercept': True, 'solver': ...",0.827189,0 days 00:00:01.566549
1,"LinearRegression(n_jobs=1, positive=True)","{'fit_intercept': True, 'n_jobs': 1, 'positive...",0.832842,0 days 00:00:00.601017
2,"KernelRidge(alpha=10, degree=1, gamma=1, kerne...","{'alpha': 10, 'degree': 1, 'gamma': 1, 'kernel...",0.838041,0 days 00:00:24.954340
3,"SGDRegressor(alpha=0.1, learning_rate='adaptiv...","{'alpha': 0.1, 'fit_intercept': True, 'learnin...",0.860281,0 days 00:00:22.987653
4,"(DecisionTreeRegressor(max_depth=2, max_featur...","{'max_depth': 2, 'n_estimators': 150}",0.884538,0 days 00:01:19.149031
5,"(DecisionTreeRegressor(max_depth=3, random_sta...","{'learning_rate': 0.01, 'n_estimators': 100}",0.884690,0 days 00:00:41.494489
6,"XGBRegressor(base_score=None, booster=None, ca...","{'max_depth': 2, 'n_estimators': 50}",0.886967,0 days 00:00:55.005960
7,([DecisionTreeRegressor(criterion='friedman_ms...,"{'max_depth': 2, 'n_estimators': 50}",0.891445,0 days 00:00:46.635239
8,DecisionTreeRegressor(max_depth=2),{'max_depth': 2},0.923438,0 days 00:00:00.258132
9,KNeighborsRegressor(n_neighbors=10),{'n_neighbors': 10},0.975806,0 days 00:00:00.226668


# Load Models from Pickle file

Load the models from the pickle to a REST API. I'm going to use _fastapi_

## Predictions

In [10]:
# ---- PREDICTIONS ----

cols = ['Wk', 'Numeric_Day', 'Numeric_Home', 'Numeric_Away', 'Numeric_Time',
       'xGHome_xGAway_1', 'xGHome_xGAway_2', 'xGHome_xGAway_3']
# display(features_away[cols].sample(1))

# Pick random number between 0 and len(data)
random_index = np.random.randint(0, len(data))

In [11]:
# display(features_away[cols][random_index:random_index+1], features_home[cols][random_index:random_index+1])
# Predict home score for match at random index
predict_res(features_home[cols][random_index:random_index+1])

'predicting ... 2 : 3'

In [12]:
predict_res(features_away[cols][random_index:random_index+1])

# The prediction is the same for both because the features are the same.

'predicting ... 2 : 3'

## Use FastApi to transform the data, load the model and do a prediction

In [13]:
root()

<coroutine object root at 0x1327d5e70>

In [14]:
print(..)

SyntaxError: invalid syntax (2448822055.py, line 1)

In [ ]:
from fastapi import FastAPI, Form
import uvicorn

app_home = FastAPI()
cols = ['Wk', 'Numeric_Day', 'Numeric_Home', 'Numeric_Away', 'Numeric_Time',
        'xGHome_xGAway_1', 'xGHome_xGAway_2', 'xGHome_xGAway_3']

class TeamStats():
    Wk: int
    Day: str
    Time: Optional[str] = None
    Home: str
    Away: str

@app_home.get("/")
async def root():
    return {"message": "Hello FastAPI!!"}

@app_home.post("/predict", response_model=Dict[str, float])  # Specify the response model type
async def predict(Wk: int = Form(...),
                Day: str = Form(...),
                Time: Optional[str] = Form(None),
                Home: str = Form(...),
                Away: str = Form(...)) -> Dict[str, float]:
    """
    Predicts the score of a match based on the team stats
    data: TeamStatistics expected
    model_type: home or away
        returns: Dict[str, float] with the predicted score
    """
    # Load the inputs into a dataframe
    data = pd.DataFrame(columns=['Wk', 'Day', 'Time', 'Home', 'Away'])
    data.loc[0] = [Wk, Day, Time, Home, Away]

    features_home, target_home = transform_data_into_features_and_targets(df=data, score='ScoreHome')
    features_away, target_away = transform_data_into_features_and_targets(df=data, score='ScoreAway')

    prediction = predict_res(features_home)

    return prediction

uvicorn.run(app_home, host="0.0.0.0", port=8000)

### ETL Process and Data Integration
_Loading of the data_ . 
__Apache Airflow__ supports a few databases: 
- SQLite _Lightweight filebased database suitable for small-scale deployments and testing_
- PostgreSQL _Relational database widely used in production environments_
- MySQL _Popular relational database widely used_
- Microsoft SQL Server _Commercial relational database widely used in enterprises_
- Oracle _Commercial relational database widely used in enterprises_
- Amazon RedShift _Cloud-based data warehouse optimized for analytics workloads_
- Google BigQuery _Cloud-based data warehouse optimized for analytics workloads_
- Apache Casssandra _Distributed No-SQL database optimized for high scalability and availability_
- Apache Hive _Data warehouse infrastructure for data summarization, querying and analytics_

I'm using SQLite because it's a small scale dataset

In [ ]:
# Load task
def load_data(matches):
    # Connect to database
    conn = sqlite3.connect('assets/spanish_matches.db')

    # Create cursor
    c = conn.cursor()

    # Create table
    c.execute("""CREATE TABLE IF NOT EXISTS matches (
        Wk INTERGER,
        Day TEXT,
        Date DATE,
        Time TIME,
        Home TEXT,
        xGHome FLOAT,
        Score TEXT,
        xGAway FLOAT,
        Away TEXT,
        xPHome FLOAT,
        xPAway FLOAT,
        ScoreHome INTERGER,
        ScoreAway INTERGER,
        GoalDifference INTERGER,
        Result TEXT,
        ExpectedGoalDifference FLOAT,
        Points INTERGER,
        ExpectedPoints INTERGER,
        WinPercentage FLOAT,
        TotalGoals INTERGER,
        xGRatio FLOAT
    )""")

    # Insert DataFrame records one by one.
    for i, row in matches.iterrows():
        c.execute("""INSERT INTO matches VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", (
            row['Wk'],
            row['Day'],
            row['Date'],
            row['Time'],
            row['Home'],
            row['xGHome'],
            row['Score'],
            row['xGAway'],
            row['Away'],
            row['xPHome'],
            row['xPAway'],
            row['ScoreHome'],
            row['ScoreAway'],
            row['GoalDifference'],
            row['Result'],
            row['ExpectedGoalDifference'],
            row['Points'],
            row['ExpectedPoints'],
            row['WinPercentage'],
            row['TotalGoals'],
            row['xGRatio']
        ))

    # Commit changes
    conn.commit()

    # Close cursor and connection
    c.close()
    conn.close()


In [ ]:
# Assign tasks
extract_task = PythonOperator(
    task_id='extract_data',
    python_callable=extract_data,
    dag=dag
)

transform_task = PythonOperator(
    task_id='transform_data',
    python_callable=transform_data,
    op_kwargs={'matches': '{{ ti.xcom_pull(task_ids="extract_data") }}'},
    dag=dag
)

load_task = PythonOperator(
    task_id='load_data',
    python_callable=load_data,
    op_kwargs={'matches': '{{ ti.xcom_pull(task_ids="transform_data") }}'},
    dag=dag
)

In [ ]:
!~/airflow_env/bin/airflow scheduler -D
!~/airflow_env/bin/airflow webserver -D

!~/airflow_env/bin/airflow dags list

!~/airflow_env/bin/airflow cheat-sheet

## Validation and Quality Assurance

To validate the quality of the data, I'm connecting to the database to check for null values in each column of the matches table.  
Specifically, I'll check the data type, the range anf completeness of the data

In [ ]:
# Validate data in the database and ensure the proper quality
def validate_data():
    # Connect to database
    conn = sqlite3.connect('assets/spanish_matches.db')

    # Create cursor
    c = conn.cursor()

    # Data type validation
    c.execute("""SELECT COUNT(*) FROM matches where CAST(Wk AS INTEGER) IS NULL""")
    null_count = c.fetchone()[0]
    if null_count == 0:
        print('Data type validation passed.')
    else:
        print(f'Data type validation failed with {null_count} null values.')

    # Data range validation
    c.execute("""SELECT COUNT(*) FROM matches where Wk < 1 OR Wk > 10""")
    range_count = c.fetchone()[0]
    if range_count == 0:
        print('Data range validation passed.')
    else:
        print(f'Data range validation failed with {range_count} values out of range.')

    # Data completeness validation
    c.execute("""SELECT COUNT(*) FROM matches where Wk IS NULL""")
    completeness_count = c.fetchone()[0]
    if completeness_count == 0:
        print('Data completeness validation passed.')
    else:
        print(f'Data completeness validation failed with {completeness_count} null values.')

    c.close()
    conn.close()

validate_task = PythonOperator(
    task_id='validate_data',
    python_callable=validate_data,
    op_kwargs={'matches': '{{ ti.xcom_pull(task_ids="load_data") }}'},
    dag=dag
)

# Define task dependencies
extract_task >> transform_task >> load_task >> validate_task

## Reporting and Analysis

Generate meaningful insights and reports.
- Trend analysis
- Team Performance analysis
- Team comparisons

In [ ]:
# Trend analysis
def trend_analysis():
    # Connect to database
    conn = sqlite3.connect('assets/spanish_matches.db')

    # Create cursor
    c = conn.cursor()

    # Data type validation
    c.execute("""SELECT * FROM matches""")
    matches = pd.DataFrame(c.fetchall())
    
    # Define plot function
    def plot_data():
        sns.lineplot(x='Date', y='TotlaGoals', data=matches)
        plt.title('Total Goals Scored')
        plt.xlabel('Date')
        plt.ylabel('Total Goals')
        plt.show()
    # Look at the correlation between the expected goals and the actual goals
    def calculate_correlation():
        corr_home = matches['xGHome'].corr(matches['ScoreHome'])
        print(f'Correlation between expected Goals for the Home and actual goals Home: {corr_home}')
        corr_away = matches['xGAway'].corr(matches['ScoreAway'])
        print(f'Correlation between expected Goals for the Away and actual goals Away: {corr_away}')
    
    plot_data()
    calculate_correlation()

trend_analysis_task = PythonOperator(
    task_id='trend_analysis',
    python_callable=trend_analysis,
    op_kwargs={'matches': '{{ ti.xcom_pull(task_ids="validate_data") }}'},
    dag=dag
)   

# Define task dependencies
extract_task >> transform_task >> load_task >> validate_task >> trend_analysis_task

In [ ]:
# Exectute the DAG workflow and view the results in the Airflow UI from scripts/DataPipelining.py
!~/airflow_env/bin/airflow trigger_dag 

In [ ]:
# Convert notebook to python script
!jupyter nbconvert --to script DataPipelining.ipynb --output-dir='airflow/dags/'